In [59]:
import numpy as np
import pandas as pd
import pickle
from fancyimpute import KNN

In [60]:
with open('UN_data.pickle', 'rb') as file:
    UN = pickle.load(file)
    file.close()
UN.head()

,Country,Year,gdp,pop_density,pop,gender_ratio,exp_imp,edu_prim,edu_sec,edu_tert,...,cpi,mort_child,life_exp,mort_mother,pop_gain,num_child,pop_migrants,pop_unhcr,urban,ghg_emissions
0,Afghanistan,2005,264.0,38.4,25.07,107.2,NaN,4318.819,651.453,NaN,...,71.1,89.5,56.9,821.0,4.4,7.2,0.3,159597.0,22.7,0.76
1,Afghanistan,2010,558.0,44.1,28.80,105.7,-4766.0,5279.326,2044.157,NaN,...,100.0,76.7,60.0,584.0,2.8,6.4,0.4,1199987.0,23.7,1.06
2,Afghanistan,2015,611.0,51.7,33.74,106.4,-7151.0,6333.695,2698.816,NaN,...,132.1,68.6,62.3,396.0,3.2,5.3,1.5,1421419.0,24.8,1.40
3,Albania,2005,2615.0,112.4,3.08,100.6,-1956.0,237.975,407.403,63.257,...,86.7,21.1,74.8,30.0,-0.3,1.9,2.1,92.0,46.7,2.68
4,Albania,2010,4056.0,107.3,2.94,100.7,-3053.0,224.781,355.871,122.326,...,100.0,16.8,75.6,30.0,-0.9,1.6,1.8,99.0,52.2,2.83


In [61]:
UN[UN.gdp.isna()].Country.unique()

array(['American Samoa', 'Bonaire, St. Eustatius & Saba',
       'Channel Islands', 'Falkland Islands (Malvinas)', 'Faroe Islands',
       'French Guiana', 'Gibraltar', 'Guadeloupe', 'Guam', 'Holy See',
       'Isle of Man', 'Martinique', 'Mayotte', 'Niue',
       'Northern Mariana Islands', 'Other non-specified areas', 'Réunion',
       'Saint Helena', 'Saint Pierre and Miquelon', 'South Sudan',
       'Sudan', 'TFYR of Macedonia', 'Tokelau',
       'United States Virgin Islands', 'Wallis and Futuna Islands',
       'Western Sahara', 'Serbia and Monten. [former]',
       'CÃ´te dâ\x80\x99Ivoire', 'CuraÃ§ao', 'European Union (EU)',
       'Guernsey', 'Jersey', 'Ascension', 'Bonaire', 'Åland Islands',
       "CÃ´te d'Ivoire", 'RÃ©union', 'Swaziland'], dtype=object)

In [62]:
print(UN.shape)
UN2 = UN[~UN.gdp.isna()]  # remove rows that don't have the response
print(UN2.shape)

(744, 36)
(638, 36)


In [63]:
UN2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 638 entries, 0 to 637
Data columns (total 36 columns):
Country              638 non-null object
Year                 638 non-null int64
gdp                  638 non-null float64
pop_density          623 non-null float64
pop                  623 non-null float64
gender_ratio         597 non-null float64
exp_imp              603 non-null float64
edu_prim             521 non-null float64
edu_sec              458 non-null float64
edu_tert             396 non-null float64
gva_ag               626 non-null float64
gva_indst            632 non-null float64
gva_servc            632 non-null float64
labor_part           563 non-null float64
unemp                564 non-null float64
energy_prod          600 non-null float64
energy_supp          622 non-null float64
internet             613 non-null float64
tourism              552 non-null float64
patents              279 non-null float64
health               560 non-null float64
edu_ratio_gb_pri

In [64]:
UN2.describe()

,Year,gdp,pop_density,pop,gender_ratio,exp_imp,edu_prim,edu_sec,edu_tert,gva_ag,...,cpi,mort_child,life_exp,mort_mother,pop_gain,num_child,pop_migrants,pop_unhcr,urban,ghg_emissions
count,638.000000,638.000000,623.000000,623.000000,597.000000,603.000000,521.000000,458.000000,396.000000,626.000000,...,535.000000,561.000000,562.000000,538.000000,616.000000,570.000000,623.000000,5.050000e+02,625.000000,545.000000
mean,2010.000000,14317.564263,426.525843,33.053451,101.172864,-783.381426,3204.126727,2988.881510,1158.171573,12.159376,...,103.771028,30.932264,69.407473,192.096654,1.484578,2.987544,10.787961,2.214996e+05,58.010240,7.216862
std,4.079277,22129.112090,2199.504823,131.159506,19.627784,67678.942843,11193.537676,11251.719557,3932.023675,12.609986,...,44.382351,27.916723,9.051645,269.081024,1.502920,1.581373,15.716854,6.669117e+05,24.254861,8.473719
min,2005.000000,91.000000,0.100000,0.000000,84.300000,-827981.000000,0.509000,0.298000,0.005000,0.021500,...,39.200000,1.600000,41.300000,3.000000,-2.300000,0.800000,0.100000,1.000000e+00,9.000000,0.260000
25%,2005.000000,1367.750000,33.650000,0.875000,96.100000,-2976.500000,108.727000,80.108250,47.543250,2.478950,...,88.150000,8.200000,63.875000,15.000000,0.500000,1.800000,1.400000,1.074000e+03,38.200000,1.890000
50%,2010.000000,4744.000000,80.100000,6.160000,98.500000,-591.000000,534.932000,512.721500,195.285500,7.760100,...,100.000000,20.400000,71.800000,59.000000,1.300000,2.400000,4.000000,1.423700e+04,58.000000,4.670000
75%,2015.000000,18186.750000,198.600000,20.815000,101.200000,658.000000,2720.989000,2008.292500,574.846000,18.847700,...,108.600000,49.500000,75.675000,287.500000,2.300000,4.000000,12.650000,1.438510e+05,77.200000,9.230000
max,2015.000000,167594.000000,25709.400000,1397.030000,313.700000,593904.000000,138518.028000,129542.058000,43367.394000,70.327500,...,772.000000,134.900000,83.400000,1986.000000,14.400000,7.700000,88.500000,7.946539e+06,100.000000,76.410000


In [65]:
# Drop features with many missing values
UN2 = UN2.dropna(axis = 1, thresh = 500)

# For each feature and country, when only one or two values are missing (value from 
# one or two years), fill NaN with mean value from the other years for that country
for col in UN2.columns[2:]:
    UN2[col] = UN2.groupby('Country')[col].transform(lambda x: x.fillna(x.mean()))
UN2.head(10)

,Country,Year,gdp,pop_density,pop,gender_ratio,exp_imp,edu_prim,gva_ag,gva_indst,...,cpi,mort_child,life_exp,mort_mother,pop_gain,num_child,pop_migrants,pop_unhcr,urban,ghg_emissions
0,Afghanistan,2005,264.0,38.4,25.07,107.2,-5958.5,4318.819,35.2479,25.9585,...,71.1,89.5,56.9,821.0,4.4,7.2,0.3,159597.0,22.7,0.76
1,Afghanistan,2010,558.0,44.1,28.80,105.7,-4766.0,5279.326,28.8094,21.3476,...,100.0,76.7,60.0,584.0,2.8,6.4,0.4,1199987.0,23.7,1.06
2,Afghanistan,2015,611.0,51.7,33.74,106.4,-7151.0,6333.695,22.6614,22.6744,...,132.1,68.6,62.3,396.0,3.2,5.3,1.5,1421419.0,24.8,1.40
3,Albania,2005,2615.0,112.4,3.08,100.6,-1956.0,237.975,21.5420,28.6709,...,86.7,21.1,74.8,30.0,-0.3,1.9,2.1,92.0,46.7,2.68
4,Albania,2010,4056.0,107.3,2.94,100.7,-3053.0,224.781,20.6582,28.6905,...,100.0,16.8,75.6,30.0,-0.9,1.6,1.8,99.0,52.2,2.83
5,Albania,2015,3895.0,106.7,2.92,102.0,-2391.0,188.371,22.5215,24.7798,...,111.4,14.6,77.7,29.0,-0.1,1.7,1.8,8098.0,57.4,2.85
6,Algeria,2005,3100.0,14.0,33.29,102.3,25645.0,4361.744,8.0130,59.7320,...,81.8,37.3,71.5,148.0,1.3,2.4,0.6,94407.0,63.8,4.04
7,Algeria,2010,4463.0,15.2,36.12,101.9,16051.0,3312.440,8.6209,51.4168,...,100.0,32.6,73.9,147.0,1.6,2.7,0.6,94448.0,67.5,4.58
8,Algeria,2015,4163.0,16.7,39.87,102.0,-17007.0,3925.429,12.0836,37.2842,...,126.7,27.7,75.3,140.0,2.0,3.0,0.6,100036.0,70.8,4.88
9,Andorra,2005,41281.0,167.8,0.08,109.0,-1653.0,4.085,0.4433,17.3054,...,89.6,NaN,NaN,NaN,3.7,1.2,63.8,NaN,90.3,NaN


In [66]:
UN2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 638 entries, 0 to 637
Data columns (total 30 columns):
Country          638 non-null object
Year             638 non-null int64
gdp              638 non-null float64
pop_density      625 non-null float64
pop              625 non-null float64
gender_ratio     607 non-null float64
exp_imp          608 non-null float64
edu_prim         588 non-null float64
gva_ag           626 non-null float64
gva_indst        632 non-null float64
gva_servc        632 non-null float64
labor_part       583 non-null float64
unemp            586 non-null float64
energy_prod      614 non-null float64
energy_supp      629 non-null float64
internet         619 non-null float64
tourism          595 non-null float64
health           563 non-null float64
parliam_women    573 non-null float64
food_prod        598 non-null float64
cpi              568 non-null float64
mort_child       565 non-null float64
life_exp         568 non-null float64
mort_mother      538 non

In [67]:
# remove countries with a lot of missing features (generally islands)
UN2 = UN2[~UN2['Country'].isin(['Andorra', 'Anguilla', 'Aruba', 'Bermuda', 'British Virgin Islands', 
                           'Cayman Islands', 'China, Hong Kong SAR', 'China, Macao SAR', 
                           'Cook Islands', 'Curaçao', 'Ivory Coast', 'Dominica', 'Eswatini',
                           'French Polynesia', 'Greenland', 'Kosovo', 'Liechtenstein',
                           'Marshall Islands', 'Monaco', 'Montserrat', 'Nauru', 
                            'Netherlands Antilles [former]', 'North Macedonia', 'Puerto Rico',
                           'Saint Kitts and Nevis', 'San Marino', 'Sint Maarten (Dutch part)',
                           'Somalia', 'Sudan [former]', 'Turks and Caicos Islands', 'Tuvalu',
                           'Zanzibar'])]

In [68]:
# for each country and feature, if all values missing for that country (all 3
# years), impute NaN with kNN using 3 most similar countries (k = 9)
UN2_num = UN2.select_dtypes(include=[np.float])
UN3 = pd.DataFrame(data = KNN(9).fit_transform(UN2_num), 
                   columns = UN2_num.columns, index = UN2_num.index)
UN3[UN3.isna().any(axis=1)]

Imputing row 1/544 with 0 missing, elapsed time: 0.096
Imputing row 101/544 with 0 missing, elapsed time: 0.097
Imputing row 201/544 with 0 missing, elapsed time: 0.098
Imputing row 301/544 with 0 missing, elapsed time: 0.099
Imputing row 401/544 with 0 missing, elapsed time: 0.099
Imputing row 501/544 with 3 missing, elapsed time: 0.100


,gdp,pop_density,pop,gender_ratio,exp_imp,edu_prim,gva_ag,gva_indst,gva_servc,labor_part,...,cpi,mort_child,life_exp,mort_mother,pop_gain,num_child,pop_migrants,pop_unhcr,urban,ghg_emissions


In [69]:
# rearrange columns
UN3 = pd.concat([UN2[['Country', 'Year']], UN3], axis = 1)
UN3.head()

,Country,Year,gdp,pop_density,pop,gender_ratio,exp_imp,edu_prim,gva_ag,gva_indst,...,cpi,mort_child,life_exp,mort_mother,pop_gain,num_child,pop_migrants,pop_unhcr,urban,ghg_emissions
0,Afghanistan,2005,264.0,38.4,25.07,107.2,-5958.5,4318.819,35.2479,25.9585,...,71.1,89.5,56.9,821.0,4.4,7.2,0.3,159597.0,22.7,0.76
1,Afghanistan,2010,558.0,44.1,28.80,105.7,-4766.0,5279.326,28.8094,21.3476,...,100.0,76.7,60.0,584.0,2.8,6.4,0.4,1199987.0,23.7,1.06
2,Afghanistan,2015,611.0,51.7,33.74,106.4,-7151.0,6333.695,22.6614,22.6744,...,132.1,68.6,62.3,396.0,3.2,5.3,1.5,1421419.0,24.8,1.40
3,Albania,2005,2615.0,112.4,3.08,100.6,-1956.0,237.975,21.5420,28.6709,...,86.7,21.1,74.8,30.0,-0.3,1.9,2.1,92.0,46.7,2.68
4,Albania,2010,4056.0,107.3,2.94,100.7,-3053.0,224.781,20.6582,28.6905,...,100.0,16.8,75.6,30.0,-0.9,1.6,1.8,99.0,52.2,2.83


In [70]:
# normalize 'exp_imp', 'edu_prim', 'energy_prod', 'tourism', and 'pop_unhcr'
# by population (size) of country (other features already normalized)
UN3['exp_imp'] = UN3['exp_imp'] / UN3['pop']
UN3['edu_prim'] = UN3['edu_prim'] / UN3['pop']
UN3['energy_prod'] = UN3['energy_prod'] / UN3['pop']
UN3['tourism'] = UN3['tourism'] / UN3['pop']
UN3['pop_unhcr'] = UN3['pop_unhcr'] / UN3['pop']
UN3.head()

,Country,Year,gdp,pop_density,pop,gender_ratio,exp_imp,edu_prim,gva_ag,gva_indst,...,cpi,mort_child,life_exp,mort_mother,pop_gain,num_child,pop_migrants,pop_unhcr,urban,ghg_emissions
0,Afghanistan,2005,264.0,38.4,25.07,107.2,-237.674511,172.270403,35.2479,25.9585,...,71.1,89.5,56.9,821.0,4.4,7.2,0.3,6366.055046,22.7,0.76
1,Afghanistan,2010,558.0,44.1,28.80,105.7,-165.486111,183.309931,28.8094,21.3476,...,100.0,76.7,60.0,584.0,2.8,6.4,0.4,41666.215278,23.7,1.06
2,Afghanistan,2015,611.0,51.7,33.74,106.4,-211.944280,187.720658,22.6614,22.6744,...,132.1,68.6,62.3,396.0,3.2,5.3,1.5,42128.601067,24.8,1.40
3,Albania,2005,2615.0,112.4,3.08,100.6,-635.064935,77.264610,21.5420,28.6709,...,86.7,21.1,74.8,30.0,-0.3,1.9,2.1,29.870130,46.7,2.68
4,Albania,2010,4056.0,107.3,2.94,100.7,-1038.435374,76.456122,20.6582,28.6905,...,100.0,16.8,75.6,30.0,-0.9,1.6,1.8,33.673469,52.2,2.83


In [71]:
UN3.to_pickle('UN_data_clean.pickle')